### Carga de librerias necesarias para el analisis multivariante

In [42]:

#==============================================================================
# EJEMPLO DEL MÉTODO DE REGRESIÓN LINEAL SIMPLE USANDO LAS LIBRERÍAS 
# SKLEARN Y STATSMODELS (EJEMPLO 1)
#==============================================================================

# Cargar librerías
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
import itertools
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.express as px

### Se carga el dataset 
# Walmart Sales 


El conjunto de datos de ventas de Walmart contiene información histórica de 45 tiendas, incluyendo datos semanales de ventas y factores económicos y regionales que pueden influir en ellas. El objetivo es utilizar esta información para predecir las ventas semanales (Weekly_Sales) basándose en características como la temperatura, el precio del combustible, el índice de precios al consumidor (CPI) y la tasa de desempleo.

A continuación, se presenta un script de Python que implementa y compara tres modelos de Machine Learning (Regresión Lineal con regularización Ridge, Random Forest y XGBoost) para predecir las ventas.

| Variable         | Tipo         | Descripción                                                                                   | Valores / Ejemplo                  |
|------------------|--------------|----------------------------------------------------------------------------------------------|------------------------------------|
| **Store**        | Numérica     | 🏬 Número único de identificación de cada tienda (del 1 al 45).                              | 1, 2, ..., 45                      |
| **Weekly_Sales** | Numérica     | 💵 Total de ventas registradas en una tienda durante una semana específica.<br>🎯 *Variable objetivo a predecir.* | 1,643,690.90                       |
| **Holiday_Flag** | Categórica Binaria | 🎉 Indica si la semana incluye un día festivo importante (Super Bowl, Labor Day, Thanksgiving, Christmas). | 1 = Semana de feriado<br>0 = No feriado |
| **Temperature**  | Numérica     | 🌡️ Temperatura promedio en la región de la tienda durante la semana (°F).                    | 42.31                              |
| **Fuel_Price**   | Numérica     | ⛽ Costo promedio del combustible en la región de la tienda.                                  | 2.572                              |
| **CPI**          | Numérica     | 📈 Índice de Precios al Consumidor (inflación y poder adquisitivo en la región).              | 211.10                             |
| **Unemployment** | Numérica     | 📉 Tasa de desempleo en la región de la tienda.                                               | 8.106                              |
| **Year**         | Numérica     | 📅 Año en que se registraron las ventas (extraído de `Date`).                                | 2010, 2011, ...                    |
| **Month**        | Numérica     | 🗓️ Mes en que se registraron las ventas (1 a 12, extraído de `Date`).                        | 2, 3, ...                          |
| **Week**         | Numérica     | 📆 Número de la semana del año (1 a 52, extraído de `Date`).                                 | 5, 6, ...                          |
| **Day**          | Numérica     | 📅 Día del mes en que finaliza la semana de ventas (extraído de `Date`).                     | 5, 12, ...                         |
| **Season**       | Categórica   | 🍂 Estación del año correspondiente al mes de la venta.<br>*Captura patrones estacionales.*   | Invierno, Primavera, Verano, Otoño |
| **Store_Tier**   | Categórica   | 🏆 Nivel asignado a cada tienda según su rendimiento de ventas promedio histórico.<br>*Agrupa tiendas con comportamiento similar.* | Alto: Tercio superior<br>Medio: Tercio medio<br>Bajo: Tercio inferior |


### Carga el DataSet

In [30]:
df = pd.read_csv('./Walmart.csv') # Cargar dataset
df.head()


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day,Season,Store_Tier
0,1,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,5,Invierno,Alto
1,1,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,12,Invierno,Alto
2,1,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,19,Invierno,Alto
3,1,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,26,Invierno,Alto
4,1,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,5,Primavera,Alto


In [31]:
df = pd.get_dummies(df, columns=['Season', 'Store_Tier'], drop_first=True)

print("Visualización del DataFrame después de One-Hot Encoding:")
print(df.head())

# Verificación de valores nulos (NaN)
print("Valores nulos en el dataset después de la limpieza:")
print(df.isnull().sum())

Visualización del DataFrame después de One-Hot Encoding:
   Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0      1    1643690.90             0        42.31       2.572  211.096358   
1      1    1641957.44             1        38.51       2.548  211.242170   
2      1    1611968.17             0        39.93       2.514  211.289143   
3      1    1409727.59             0        46.63       2.561  211.319643   
4      1    1554806.68             0        46.50       2.625  211.350143   

   Unemployment  Year  Month  Week  Day  Season_Otoño  Season_Primavera  \
0         8.106  2010      2     5    5         False             False   
1         8.106  2010      2     6   12         False             False   
2         8.106  2010      2     7   19         False             False   
3         8.106  2010      2     8   26         False             False   
4         8.106  2010      3     9    5         False              True   

   Season_Verano  Store_Tier_

### Segmenta las variables independientes (X) y la variable dependiente (y)

In [32]:
# Segmenta las variables independientes (X) y la variable dependiente (y)
X = df.drop('Weekly_Sales', axis=1)
y = df['Weekly_Sales']

 ### Regresión Lineal Múltiple (Mínimos Cuadrados Ordinarios - OLS) 

In [34]:
# Convertir columnas booleanas a int para evitar errores con statsmodels
X_OLS = sm.add_constant(X)
X_OLS = X_OLS.astype({col: int for col in X_OLS.select_dtypes('bool').columns})

# Ajustar el modelo OLS
model_ols = sm.OLS(y, X_OLS).fit()

# Imprimir el resumen estadístico completo
print(model_ols.summary())

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     1950.
Date:                Tue, 14 Oct 2025   Prob (F-statistic):               0.00
Time:                        21:28:41   Log-Likelihood:                -88834.
No. Observations:                6435   AIC:                         1.777e+05
Df Residuals:                    6419   BIC:                         1.778e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -9.817e+06   3.18e+07  

# Dividir el dataset en entrenamiento y prueba

In [36]:
# Dividir el dataset en entrenamiento y prueba de forma estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### Escalar variables

In [37]:
# Escalar variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Crear Modelos de clasificacion para comparacion de rendimiento


In [38]:
# --- Regresión Lineal (Ridge) ---
print("-" * 30)
print("Optimizando y Entrenando Regresión Ridge...")
ridge_params = {'alpha': [0.1, 1.0, 10.0, 50.0, 100.0]}
ridge_grid = GridSearchCV(Ridge(), ridge_params, cv=5, scoring='r2', n_jobs=-1)
ridge_grid.fit(X_train, y_train)
model_lr = ridge_grid.best_estimator_

y_pred_lr = model_lr.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("\nRegresión Lineal (Ridge) optimizada:")
print(f"  Mejor alpha: {ridge_grid.best_params_['alpha']}")
print(f"  RMSE: ${rmse_lr:,.2f}")
print(f"  MAE: ${mae_lr:,.2f}")
print(f"  R²: {r2_lr:.3f}")

------------------------------
Optimizando y Entrenando Regresión Ridge...

Regresión Lineal (Ridge) optimizada:
  Mejor alpha: 0.1
  RMSE: $240,606.85
  MAE: $182,570.64
  R²: 0.820


In [39]:
# --- Random Forest ---
print("-" * 30)
print("Optimizando y Entrenando Random Forest...")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_leaf': [2, 4]
}
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42, n_jobs=-1), rf_params, cv=3, scoring='r2', verbose=1)
rf_grid.fit(X_train, y_train)
model_rf = rf_grid.best_estimator_

y_pred_rf = model_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\nRandom Forest optimizado:")
print(f"  Mejores Parámetros: {rf_grid.best_params_}")
print(f"  RMSE: ${rmse_rf:,.2f}")
print(f"  MAE: ${mae_rf:,.2f}")
print(f"  R²: {r2_rf:.3f}")


------------------------------
Optimizando y Entrenando Random Forest...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

Random Forest optimizado:
  Mejores Parámetros: {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 200}
  RMSE: $87,775.13
  MAE: $50,157.80
  R²: 0.976


In [40]:
# --- XGBoost ---
print("-" * 30)
print("Optimizando y Entrenando XGBoost...")
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1]
}
xgb_grid = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1), xgb_params, cv=3, scoring='r2', verbose=1)
xgb_grid.fit(X_train, y_train)
model_xgb = xgb_grid.best_estimator_

y_pred_xgb = model_xgb.predict(X_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("\nXGBoost optimizado:")
print(f"  Mejores Parámetros: {xgb_grid.best_params_}")
print(f"  RMSE: ${rmse_xgb:,.2f}")
print(f"  MAE: ${mae_xgb:,.2f}")
print(f"  R²: {r2_xgb:.3f}")
print("-" * 30)


------------------------------
Optimizando y Entrenando XGBoost...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

XGBoost optimizado:
  Mejores Parámetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
  RMSE: $69,571.45
  MAE: $43,098.28
  R²: 0.985
------------------------------


#  --- Comparación de Modelos ---


In [41]:
# --- Comparación de Modelos ---
# Crear DataFrame para comparar los modelos
models = pd.DataFrame({
    'Model': ['Ridge Regression', 'Random Forest', 'XGBoost'],
    'RMSE': [rmse_lr, rmse_rf, rmse_xgb],
    'MAE': [mae_lr, mae_rf, mae_xgb],
    'R²': [r2_lr, r2_rf, r2_xgb]
})

print("\nTabla Comparativa de Métricas:")
print(models.to_string())

# Gráfico de Barras para R²
fig_r2 = px.bar(models.sort_values('R²', ascending=True),
                x='R²', y='Model', color='Model',
                template='plotly_dark', title='Comparación de R² por Modelo',
                text='R²',
                labels={'R²': 'R² Score', 'Model': 'Modelo'})
fig_r2.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig_r2.update_layout(xaxis=dict(range=[0, 1]))
fig_r2.show()

# Gráfico de Barras para RMSE
fig_rmse = px.bar(models.sort_values('RMSE', ascending=False),
                  x='RMSE', y='Model', color='Model',
                  template='plotly_dark', title='Comparación de RMSE por Modelo',
                  text='RMSE',
                  labels={'RMSE': 'RMSE ($)', 'Model': 'Modelo'})
fig_rmse.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig_rmse.show()


Tabla Comparativa de Métricas:
              Model           RMSE            MAE        R²
0  Ridge Regression  240606.845801  182570.644322  0.820299
1     Random Forest   87775.131651   50157.803060  0.976085
2           XGBoost   69571.446989   43098.279406  0.984976


In [ ]:
predictors = X_OLS.columns.drop('const')

# Almacenar resultados
results_aic = []
results_r2 = []

# Iterar sobre todas las combinaciones de 2 variables
for combo in itertools.combinations(predictors, 2):
    X_subset = X_OLS[['const'] + list(combo)]
    model_subset = sm.OLS(y, X_subset).fit()
    results_aic.append((combo, model_subset.aic))
    results_r2.append((combo, model_subset.rsquared))

# --- Criterio AIC ---
best_model_aic = min(results_aic, key=lambda item: item[1])
print("\\n--- Mejor Modelo según Criterio AIC ---")
print(f"Variables: {best_model_aic[0]}")
print(f"AIC: {best_model_aic[1]:.2f}")

# --- Criterio R-cuadrado ---
best_model_r2 = max(results_r2, key=lambda item: item[1])
print("\\n--- Mejor Modelo según Criterio R-cuadrado ---")
print(f"Variables: {best_model_r2[0]}")
print(f"R²: {best_model_r2[1]:.3f}")

\n--- Mejor Modelo según Criterio AIC ---
Variables: ('Store_Tier_Bajo', 'Store_Tier_Medio')
AIC: 178679.79
\n--- Mejor Modelo según Criterio R-cuadrado ---
Variables: ('Store_Tier_Bajo', 'Store_Tier_Medio')
R²: 0.790


In [44]:
# Realizar la prueba de Breusch-Pagan con los residuos del modelo completo
bp_test = het_breuschpagan(model_ols.resid, model_ols.model.exog)

labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
results = dict(zip(labels, bp_test))

print("\\n--- Prueba de Homocedasticidad (Breusch-Pagan) ---")
print(f"P-valor de la prueba: {results['p-value']:.4f}")

if results['p-value'] < 0.05:
    print("Conclusión: Se rechaza la hipótesis nula. Hay evidencia de heterocedasticidad.")
else:
    print("Conclusión: No se rechaza la hipótesis nula. No hay evidencia de heterocedasticidad.")

\n--- Prueba de Homocedasticidad (Breusch-Pagan) ---
P-valor de la prueba: 0.0000
Conclusión: Se rechaza la hipótesis nula. Hay evidencia de heterocedasticidad.
